In [10]:
# Importing libraries required
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import seaborn as sns # Data visualization
import matplotlib.pyplot as plt # Data visualization

# Data analysis and ML Library
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Lasso # Lasso regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression # Logistic regression
from sklearn.svm import SVC # Support vector machine classifier
from sklearn.tree import DecisionTreeClassifier # Decision tree classifier
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bayes
from sklearn.neighbors import KNeighborsClassifier# K Nearset Neighbors
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score,f1_score,log_loss,roc_curve,classification_report, confusion_matrix

# Handle Imbalanced dataset
from imblearn.over_sampling import SMOTE

# Print html elements
from IPython.display import Markdown

# Ignore warning messages
import warnings
warnings.filterwarnings('ignore')

#To make working with geospatial data in python easier
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

plt.style.use('ggplot')

In [11]:
df = pd.read_csv("./data/ca_ev_registrations_public.csv")
df.head()

,Vehicle ID,County GEOID,Registration Valid Date,DMV ID,DMV Snapshot,Registration Expiration Date,State Abbreviation,Geography,Vehicle Name
0,CA-002-03597\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),NaN,CA,County,Chevrolet Volt
1,CA-002-03598\r,06105,2011-01-01,2,CA Registration Data from CA (12/31/2011),NaN,CA,County,Nissan Leaf
2,CA-002-03599\r,06103,2011-01-01,2,CA Registration Data from CA (12/31/2011),NaN,CA,County,Chevrolet Volt
3,CA-002-03600\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),NaN,CA,County,Tesla Roadster
4,CA-002-03601\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),NaN,CA,County,Tesla Roadster


In [12]:
df.shape

(2542443, 9)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542443 entries, 0 to 2542442
Data columns (total 9 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Vehicle ID                    object 
 1   County GEOID                  object 
 2   Registration Valid Date       object 
 3   DMV ID                        int64  
 4   DMV Snapshot                  object 
 5   Registration Expiration Date  float64
 6   State Abbreviation            object 
 7   Geography                     object 
 8   Vehicle Name                  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 174.6+ MB


In [14]:
df.describe()

,DMV ID,Registration Expiration Date
count,2542443.000,0.000
mean,8.874,NaN
std,1.966,NaN
min,1.000,NaN
25%,8.000,NaN
50%,9.000,NaN
75%,10.000,NaN
max,11.000,NaN


In [15]:
df = df.drop(columns = ["Registration Expiration Date"])
df.head()

,Vehicle ID,County GEOID,Registration Valid Date,DMV ID,DMV Snapshot,State Abbreviation,Geography,Vehicle Name
0,CA-002-03597\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt
1,CA-002-03598\r,06105,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Nissan Leaf
2,CA-002-03599\r,06103,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt
3,CA-002-03600\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster
4,CA-002-03601\r,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster


In [34]:
df["Registration Valid Date"].value_counts()

Registration Valid Date
2020-01-01    628473
2019-01-01    559969
2018-01-01    448567
2017-01-01    322762
2016-01-01    230171
2015-01-01    166168
2014-01-01    108939
2013-01-01     52427
2012-01-01     18356
2011-01-01      5857
2010-01-01       754
Name: count, dtype: int64

In [38]:
df['Registration Valid Date'] = df['Registration Valid Date'].astype("datetime64[ns]")


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542443 entries, 0 to 2542442
Data columns (total 7 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   County GEOID             object        
 1   Registration Valid Date  datetime64[ns]
 2   DMV ID                   int64         
 3   DMV Snapshot             object        
 4   State Abbreviation       object        
 5   Geography                object        
 6   Vehicle Name             object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 135.8+ MB


In [17]:
df.duplicated().sum()

0

In [40]:
df.head()

,County GEOID,Registration Valid Date,DMV ID,DMV Snapshot,State Abbreviation,Geography,Vehicle Name
0,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt
1,06105,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Nissan Leaf
2,06103,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt
3,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster
4,06099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster


In [41]:
df = df.drop(columns = ["Vehicle ID"])
df.head()

KeyError: "['Vehicle ID'] not found in axis"

In [42]:
df.describe(include = 'object')

,County GEOID,DMV Snapshot,State Abbreviation,Geography,Vehicle Name
count,2542443,2542443,2542443,2542443,2542443
unique,117,11,1,1,82
top,6037,CA Registration Data from CA (12/31/2020),CA,County,Chevrolet Volt
freq,465061,628473,2542443,2542443,349602


In [43]:
df.describe(include = "number")

,DMV ID
count,2542443.000
mean,8.874
std,1.966
min,1.000
25%,8.000
50%,9.000
75%,10.000
max,11.000


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542443 entries, 0 to 2542442
Data columns (total 7 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   County GEOID             object        
 1   Registration Valid Date  datetime64[ns]
 2   DMV ID                   int64         
 3   DMV Snapshot             object        
 4   State Abbreviation       object        
 5   Geography                object        
 6   Vehicle Name             object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 135.8+ MB


In [59]:
df = df[df["County GEOID"] != "Unknown"]

In [60]:
df["County GEOID"].isnull().sum()

0

In [63]:
df["County GEOID"] = df["County GEOID"].astype("int")

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2522939 entries, 0 to 2542442
Data columns (total 7 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   County GEOID             int32         
 1   Registration Valid Date  datetime64[ns]
 2   DMV ID                   int64         
 3   DMV Snapshot             object        
 4   State Abbreviation       object        
 5   Geography                object        
 6   Vehicle Name             object        
dtypes: datetime64[ns](1), int32(1), int64(1), object(4)
memory usage: 144.4+ MB


In [68]:
df["DMV Snapshot"]

0          CA Registration Data from CA (12/31/2011)
1          CA Registration Data from CA (12/31/2011)
2          CA Registration Data from CA (12/31/2011)
3          CA Registration Data from CA (12/31/2011)
4          CA Registration Data from CA (12/31/2011)
                             ...                    
2542438    CA Registration Data from CA (12/31/2020)
2542439    CA Registration Data from CA (12/31/2020)
2542440    CA Registration Data from CA (12/31/2020)
2542441    CA Registration Data from CA (12/31/2020)
2542442    CA Registration Data from CA (12/31/2020)
Name: DMV Snapshot, Length: 2522939, dtype: object

In [76]:
import re
df['DMV Source date'] = pd.to_datetime(df['DMV Snapshot'].apply(lambda text: re.search('[0-9]{,2}\/[0-9]{,2}\/[0-9]{4}', text)[0]))

In [77]:
df.head()

,County GEOID,Registration Valid Date,DMV ID,DMV Snapshot,State Abbreviation,Geography,Vehicle Name,DMV Source date
0,6099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt,2011-12-31
1,6105,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Nissan Leaf,2011-12-31
2,6103,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Chevrolet Volt,2011-12-31
3,6099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster,2011-12-31
4,6099,2011-01-01,2,CA Registration Data from CA (12/31/2011),CA,County,Tesla Roadster,2011-12-31


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2522939 entries, 0 to 2542442
Data columns (total 8 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   County GEOID             int32         
 1   Registration Valid Date  datetime64[ns]
 2   DMV ID                   int64         
 3   DMV Snapshot             object        
 4   State Abbreviation       object        
 5   Geography                object        
 6   Vehicle Name             object        
 7   DMV Source date          datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(4)
memory usage: 163.6+ MB


In [79]:
cleaned_df = df.to_csv("./data/ca_ev_registrations_public_cleaned.csv")
cleaned_df